In [1]:
import os, sys, codecs, time, datetime
import fileinput
import os.path
from io import StringIO
from subprocess import call


from docopt import docopt
from propsde.applications.viz_tree import DepTreeVisualizer

import propsde.applications.run as run
import sys
if sys.version_info[0] >= 3:
    unicode = str

In [2]:
import pickle

In [3]:
with open('polizeiberichte.pkl', 'rb') as f:
    data = pickle.load(f, encoding="utf8")

In [4]:
text_example = data[72]["text1"]
print(text_example)

Der Polizeiliche Staatsschutz beim Landeskriminalamt ermittelt wegen gemeinschädlicher Sachbeschädigung an einem jüdischen Mahnmal in Tiergarten. Passanten hatten heute früh gegen 8.50 Uhr gelbe Schmierereien an der Gedenktafel in der Levetzowstraße entdeckt und die Polizei alarmiert.


In [5]:
text_example = text_example.replace(". ", ".\n")

In [6]:
print(text_example)

Der Polizeiliche Staatsschutz beim Landeskriminalamt ermittelt wegen gemeinschädlicher Sachbeschädigung an einem jüdischen Mahnmal in Tiergarten.
Passanten hatten heute früh gegen 8.50 Uhr gelbe Schmierereien an der Gedenktafel in der Levetzowstraße entdeckt und die Polizei alarmiert.


In [7]:
file = codecs.open("examples.txt", encoding='utf8')
gs = run.parseSentences(file)

running mate-tools
lemmatizing
parsing + tagging
collapsing
java -jar ext/org.jobimtext.collapsing.jar -i tmp/parsed.conll06 -o tmp -sf -l de -r ext/resources/german_modified.txt -f c -np -nt


In [8]:
for g,tree in gs: 
    for prop in g.getPropositions('pdf'):
        curProp = '{0}:({1})'.format(prop.pred,
                                      ", ".join([rel + ":" + arg for rel,arg in sorted(prop.args,key=lambda rel:prop.rel_order(rel[0]))]))
        print(curProp)

fahren:(subj:Der regierende Bürgermeister , prep_zu:Rathaus )
gehen:(subj:er , prep_in:roten Rathaus , prep_in:den Sitzungssaal )
liegen:(subj:Der alte Sitzungssaal , prep_neben:dem Büro von Michael Müller )


In [9]:
prop = g.getPropositions('pdf')[0]
print(prop.args)
print(prop.pred)

[['subj', 'Der alte Sitzungssaal '], ['prep_neben', 'dem Büro von Michael Müller ']]
liegen


In [16]:
test = gs[0][0]
print(test)

Der regierende Bürgermeister fährt zum Rathaus .
1	2,regierende	ADJA	0	0	mod,2
2	3,Bürgermeister	NN	0	0	subj,3
3	4,fährt	VVFIN	1	1	
4	6,Rathaus	NN	0	0	prep_zu,3



In [120]:
from itertools import compress

def _map_article_of_subj_to_pronoun(arg):
    personal_pronouns = {"der": "er", "das": "es", "die": "sie"}
    if "subj" in arg:
        words = arg[1].lower().split()
        pronoun_place = list(word in personal_pronouns for word in words)
        pronoun = list(compress(words, pronoun_place))
        if len(pronoun) == 1:
            current_subj = (personal_pronouns[pronoun[0]], arg[1])
            return current_subj
    return None
    
    

In [166]:
current_subj = []

for i, (g, tree) in enumerate(gs):
    current_subj.append([])
    for prop in g.getPropositions('pdf'):
        for arg in prop.args:
            if len(current_subj) > 0:
                if current_subj[0] in words:
                    print("Test")
            
            words = arg[1].lower().split()
            if i > 0 and len(current_subj[i-1]) > 0 and current_subj[i-1][0][0] in words:
                words[words.index(current_subj[i-1][0][0])] = current_subj[i-1][0][1]
                
            article_pronoun = _map_article_of_subj_to_pronoun(arg)
            if article_pronoun:
                current_subj[i].append(article_pronoun)
            
            
            

['der', 'regierende', 'bürgermeister']
['rathaus']
['roten', 'rathaus']
['Der regierende Bürgermeister ']
['den', 'sitzungssaal']
['der', 'alte', 'sitzungssaal']
['dem', 'büro', 'von', 'michael', 'müller']


Der regierende Bürgermeister fährt zum Rathaus.
Im roten Rathaus geht er in den Sitzungssaal.
Der alte Sitzungssaal liegt neben dem Büro von Michael Müller.